** Финальное задание **
===================
**Предметная область: Игра Dota 2**
-------------------
<i> Dota 2 — многопользовательская компьютерная игра жанра MOBA. Игроки играют между собой матчи. В каждом матче участвует две команды, 5 человек в каждой. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника (трон).

<i> Существуют разные режимы игры, мы будем рассматривать режим Captain's Mode, в формате которого происходит большая часть киберспортивных мероприятий по Dota 2.

* * *

<h1 style="text-align:center;"> Решение задание </h1>

<i>Загрузка данных

In [2]:
import pandas as pd

features = pd.read_csv('features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


<h6>Описание признаков в таблице</h6>
<ul>
    <li>match_id: идентификатор матча в наборе данных</li>
    <li>start_time: время начала матча (unixtime)</li>
    <li>lobby_type: тип комнаты, в которой собираются игроки (расшифровка в dictionaries/lobbies.csv)</li>
    <li>Наборы признаков для каждого игрока (игроки команды Radiant — префикс rN, Dire — dN):
    <ul>
        <li>r1_hero: герой игрока (расшифровка в dictionaries/heroes.csv)</li>
        <li>r1_level: максимальный достигнутый уровень героя (за первые 5 игровых минут)</li>
        <li>r1_xp: максимальный полученный опыт</li>
        <li>r1_gold: достигнутая ценность героя</li>
        <li>r1_lh: число убитых юнитов</li>
        <li>r1_kills: число убитых игроков</li>
        <li>r1_deaths: число смертей героя</li>
        <li>r1_items: число купленных предметов</li>
    </ul>
    </li>
    <li>Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    <ul>
        <li>first_blood_time: игровое время первой крови</li>
        <li>first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)</li>
        <li>first_blood_player1: игрок, причастный к событию</li>
        <li>first_blood_player2: второй игрок, причастный к событию</li>
    </ul>
    </li>
    <li>Признаки для каждой команды (префиксы radiant_ и dire_)
    <ul>
        <li>radiant_bottle_time: время первого приобретения командой предмета "bottle"</li>
        <li>radiant_courier_time: время приобретения предмета "courier"</li>
        <li>radiant_flying_courier_time: время приобретения предмета "flying_courier"</li>
        <li>radiant_tpscroll_count: число предметов "tpscroll" за первые 5 минут</li>
        <li>radiant_boots_count: число предметов "boots"</li>
        <li>radiant_ward_observer_count: число предметов "ward_observer"</li>
        <li>radiant_ward_sentry_count: число предметов "ward_sentry"</li>
        <li>radiant_first_ward_time: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля</li>  
    </ul>
    </li>
    <li>Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    <ul>
        <li>duration: длительность</li>
        <li>radiant_win: 1, если победила команда Radiant, 0 — иначе</li>
        <li>Состояние башен и барраков к концу матча (см. описание полей набора данных)</li>
            <ul>
                <li>tower_status_radiant</li>
                <li>tower_status_radiant</li>
                <li>tower_status_dire</li>
                <li>barracks_status_radiant</li>
                <li>barracks_status_dire</li>
            </ul>
    </ul>
    </li>
</ul> 
* * *

<i>Удаление признаков, связанных с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [3]:
X_train = features.drop(['duration', 
                         'radiant_win',
                         'tower_status_radiant', 
                         'tower_status_dire', 
                         'barracks_status_radiant', 
                         'barracks_status_dire'], axis =1)

* * *
<i>Проверка выборки на наличие пропусков с помощью функции count().

In [4]:
import numpy as np

row, col= X_train.shape
df_index = -1
print "Features with Na:"
for i in X_train.count():
    df_index += 1
    if (i < row):
        print X_train.count()[[df_index]]

Features with Na:
first_blood_time    77677
dtype: int64
first_blood_team    77677
dtype: int64
first_blood_player1    77677
dtype: int64
first_blood_player2    53243
dtype: int64
radiant_bottle_time    81539
dtype: int64
radiant_courier_time    96538
dtype: int64
radiant_flying_courier_time    69751
dtype: int64
radiant_first_ward_time    95394
dtype: int64
dire_bottle_time    81087
dtype: int64
dire_courier_time    96554
dtype: int64
dire_flying_courier_time    71132
dtype: int64
dire_first_ward_time    95404
dtype: int64


<h6>Для любых двух признаков дайте обоснование, почему их значения могут быть пропущены.</h6>
<ul>
    <li>first_blood_time - есть пропущены значение, потому что в 79,89% случаев в первые 5 минут игры есть первая кровь</li>
    <li>radiant_bottle_time - есть пропущены значение, потому что в 83,86% случаев время первого приобретения командой предмета "bottle" в первые 5 минут игры</li>
</ul>
* * *

<i>Замена пропусков на нули с помощью функции fillna().

In [5]:
X_train.fillna(0, inplace=True)

* * *
<i>Запишем целевую переменную, она содержит переменную <b>'radiant_win'</b>.

In [9]:
y_train = features['radiant_win']

<h1>Подход 1: градиентный бустинг</h1>

<i> Градиентный бустинг не очень требователен к данным, он восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность.

<i>Оценим качество градиентного бустинга с помощью кросс-валидации.

In [16]:
import time
import datetime
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

kf = KFold(y_train.size, n_folds=5, shuffle=True, random_state=241)
n_trees = [10, 20, 30, 40, 50, 100]
score = []
for i in n_trees:
    print "Number of trees = ", i
    clf_GBC = GradientBoostingClassifier(n_estimators=i, random_state=241)
    start_time = datetime.datetime.now()
    clf_score = cross_val_score(clf_GBC, X_train, y_train, cv=kf, scoring='roc_auc')
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print "Score = ", np.mean(clf_score)
    score.append(np.mean(clf_score))
    print '============='

Number of trees =  10
Time elapsed: 0:00:22.975904
Score =  0.664387720635
Number of trees =  20
Time elapsed: 0:00:43.571226
Score =  0.682853573534
Number of trees =  30
Time elapsed: 0:01:04.549639
Score =  0.689496206059
Number of trees =  40
Time elapsed: 0:01:23.152204
Score =  0.694131121473
Number of trees =  50
Time elapsed: 0:01:42.870991
Score =  0.697454831695
Number of trees =  100
Time elapsed: 0:03:18.284774
Score =  0.706326218163


<i> Для градиентного бустинга с 30 деревьями кросс-валидация проводилась <b>1 мин 4 сек</b>, при етом качество рaвно <b>0.689496206059</b>.

 Увеличение количества деревьев также увеличивает показатель метрики AUC-ROC, приближая его к 0.7. Для достижения большего качества имеет смысл использовать более 30 деревьев. 

 Для ускорения обучения можно использовать не всю выборку, а некоторое ее подмножество — например, половину объектов. Также можно уменьшить глубину деревьев.
 * * *
 * * *

<h1>Подход 2: логистическая регрессия</h1>

<i>Линейные методы работают гораздо быстрее композиций деревьев. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

<i>Линейные алгоритмы чувствительны к масштабу признаков, поетому выполним преобработку данных.

In [20]:
from sklearn.preprocessing import StandardScaler

X_train = StandardScaler().fit_transform(X_train)

* * *
<i>Оценим качество логистической регрессии с L2-регуляризацией с помощью кросс-валидации по той же схеме, 
которая использовалась для градиентного бустинга. 

Подберем лучший параметр регуляризации (C). 

In [29]:
from sklearn.linear_model import LogisticRegression

kf2 = KFold(y_train.size, n_folds=5, shuffle=True, random_state=241)
c = [10000., 1000., 100., 10., 1., 0.1, 0.01, 0.001, 0.0001]
score = []
for i in c:
    print 'C = ', i
    clf_l2_LR = LogisticRegression(C=i, random_state=241)
    start_time = datetime.datetime.now()
    clf_score = cross_val_score(clf_l2_LR, X_train, y_train, cv=kf2, scoring='roc_auc')
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print 'Score = ', np.mean(clf_score)
    score.append(np.mean(clf_score))
    print '============='

C =  10000.0
Time elapsed: 0:00:10.597886
Score =  0.7163062573
C =  1000.0
Time elapsed: 0:00:10.547827
Score =  0.716306263653
C =  100.0
Time elapsed: 0:00:10.412064
Score =  0.716306265779
C =  10.0
Time elapsed: 0:00:10.212429
Score =  0.71630633996
C =  1.0
Time elapsed: 0:00:10.580293
Score =  0.716306583646
C =  0.1
Time elapsed: 0:00:10.320126
Score =  0.716310083653
C =  0.01
Time elapsed: 0:00:09.815678
Score =  0.716341462187
C =  0.001
Time elapsed: 0:00:07.531608
Score =  0.716180246368
C =  0.0001
Time elapsed: 0:00:04.037435
Score =  0.711250114392


<i>Наилучшее значение показателя AUC-ROC достигается при <b>C = 0.01</b> и равно <b>0.716341462187</b>. Это сравнимо с градиентным бустингом c 100 деревьям, при этом логистическая регрессия работает заметно быстрее.


* * *
Среди признаков в выборке есть категориальные, но их использование не является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. 

Уберем их из выборки, и проведем кросс-валидацию для логистической регрессии на новой выборке с параметром регуляризации = 0.01.

In [27]:
X_train_numerical = features.drop(['duration', 'radiant_win',
                                   'tower_status_radiant', 
                                   'tower_status_dire', 
                                   'barracks_status_radiant', 
                                   'barracks_status_dire',
                                   'lobby_type', 'r1_hero', 
                                   'r2_hero', 'r3_hero',
                                   'r4_hero', 'r5_hero', 
                                   'd1_hero', 'd2_hero', 
                                   'd3_hero', 'd4_hero', 
                                   'd5_hero'], axis=1)
X_train_numerical.fillna(0, inplace=True)
X_train_numerical = StandardScaler().fit_transform(X_train_numerical)

In [31]:
kf2 = KFold(y_train.size, n_folds=5, shuffle=True, random_state=241)
c = [10000., 1000., 100., 10., 1., 0.1, 0.01, 0.001, 0.0001]
score = []
for i in c:
    print 'C = ', i
    clf_l2_LR = LogisticRegression(C=i, random_state=241)
    start_time = datetime.datetime.now()
    clf_score = cross_val_score(clf_l2_LR, X_train_numerical, y_train, cv=kf2, scoring='roc_auc')
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print 'Score = ', np.mean(clf_score)
    score.append(np.mean(clf_score))
    print '============='

C =  10000.0
Time elapsed: 0:00:09.895269
Score =  0.716370530167
C =  1000.0
Time elapsed: 0:00:09.881192
Score =  0.716370530166
C =  100.0
Time elapsed: 0:00:09.779023
Score =  0.716370496271
C =  10.0
Time elapsed: 0:00:09.820793
Score =  0.716370479305
C =  1.0
Time elapsed: 0:00:09.724984
Score =  0.716370752658
C =  0.1
Time elapsed: 0:00:09.589481
Score =  0.716373784472
C =  0.01
Time elapsed: 0:00:09.149440
Score =  0.716400950653
C =  0.001
Time elapsed: 0:00:07.149222
Score =  0.716235591021
C =  0.0001
Time elapsed: 0:00:03.954122
Score =  0.711248390616


<i>Удаление категориальных признаков не повлияло на качество предсказания. Наилучшее значение показателя AUC-ROC так же достигается при <b>C = 0.01</b> и равно <b>0.716400950653</b>. 

Получается, что в предыдущей модели эти признаки никак не влияли на результат предсказания, модель смогла распознать этот шум.

* * *

<i>На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие.

Выясним из данных, сколько различных идентификаторов героев существует в данной игре.

In [33]:
hero_list = []
for i in range(1, 6):
    hero = 'r' + str(i) + '_hero'
    hero_list.append(features[hero].unique())
    hero = 'd' + str(i) + '_hero'
    hero_list.append(features[hero].unique())

N = max(pd.DataFrame(np.array(hero_list).flatten())[0].unique())
N

112

* * *
<i>Воспользуем подход <b>"мешок слов"</b> для кодирования информации о героях. 

Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не 
участвовал в матче; единице, если i-й герой играл за команду Radiant; 
минус единице, если i-й герой играл за команду Dire. 

Добавим полученные признаки к числовым, которые использовали во втором пункте данного этапа.

In [81]:
# N — количество различных героев в выборке
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in xrange(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_train_pick = np.hstack((X_train_numerical, X_pick))

<i>Проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. 

In [83]:
c = [10000., 1000., 100., 10., 1., 0.1, 0.01, 0.001, 0.0001]
score = []
for i in c:
    print 'C = ', i
    clf_l2_LR = LogisticRegression(C=i, random_state=241)
    start_time = datetime.datetime.now()
    clf_score = cross_val_score(clf_l2_LR, X_train_pick, y_train, cv=kf2, scoring='roc_auc')
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print 'Score = ', np.mean(clf_score)
    score.append(np.mean(clf_score))
    print '============='

C =  10000.0
Time elapsed: 0:00:18.357183
Score =  0.751917174307
C =  1000.0
Time elapsed: 0:00:18.500406
Score =  0.751916937051
C =  100.0
Time elapsed: 0:00:18.261905
Score =  0.751916981471
C =  10.0
Time elapsed: 0:00:18.288424
Score =  0.751917290816
C =  1.0
Time elapsed: 0:00:18.921317
Score =  0.751919741775
C =  0.1
Time elapsed: 0:00:16.790977
Score =  0.751937449549
C =  0.01
Time elapsed: 0:00:12.525427
Score =  0.751735952612
C =  0.001
Time elapsed: 0:00:07.975196
Score =  0.746296233371
C =  0.0001
Time elapsed: 0:00:03.908597
Score =  0.725022149276


<i>После добавления "мешка слов" по героям качество заметно <b>улучшилось</b>. Наилучшее значение показателя AUC-ROC достигается при <b>C = 0.1</b> и равно <b>0.751937449549</b>. 

Это объясняется тем, что вместо отсутствия данных о героях или случайного шума из id мы имеем осмысленную разреженную матрицу для построения предсказания.

* * *

<i>Построим предсказания вероятностей победы команды Radiant для тестовой выборки.

In [85]:
clf = LogisticRegression(C=0.01).fit(X_train_pick, y_train)

<i>Выполним преобработку тестовых данных.

In [86]:
X_test = pd.read_csv('features_test.csv', index_col='match_id')

X_test_pick = np.zeros((X_test.shape[0], N))
for i, match_id in enumerate(X_test.index):
    for p in xrange(5):
        X_test_pick[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_test_pick[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_test = X_test.drop(['lobby_type', 'r1_hero', 
                      'r2_hero', 'r3_hero',
                      'r4_hero', 'r5_hero', 
                      'd1_hero', 'd2_hero', 
                      'd3_hero', 'd4_hero', 
                      'd5_hero'], axis=1)

X_test.fillna(0, inplace=True)
X_test = StandardScaler().fit_transform(X_test)

X_test = np.hstack((X_test, X_test_pick))

In [87]:
pred = clf.predict_proba(X_test)[:, 1]

<i>Максимальное значение прогноза на тестовой выборке:

In [91]:
np.max(pred)

0.99598316744482462

<i>Минимальное значение прогноза на тестовой выборке:

In [93]:
np.min(pred)

0.008442628203118023